In [1]:
%load_ext autoreload 
%autoreload 2

import sys
sys.path.append('../')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from tqdm import tqdm
from warnings import filterwarnings
filterwarnings('ignore')

from src.database.weaviate_interface_v4 import WeaviateWCS
from src.database.database_utils import get_weaviate_client
from src.preprocessor.preprocessing import FileIO
from src.reranker import ReRanker
from src.evaluation.llm_evaluation import AnswerCorrectnessMetric, EvalResponse, CustomAzureOpenAI, CustomAnthropic
from src.llm.llm_interface import LLM
from src.llm.llm_utils import load_azure_openai
from src.llm.prompt_templates import (huberman_system_message, question_answering_prompt_series,
                                      create_context_blocks, generate_prompt_series)
from deepeval.test_case import LLMTestCase
from deepeval.models import DeepEvalBaseLLM
from deepeval import evaluate

import aiometer
import nest_asyncio
import asyncio 
nest_asyncio.apply()

### Load Data

In [5]:
data_path = '../data/golden_datasets/golden_256.json'
data = FileIO().load_json(data_path)
queries = list(data['queries'].values())

### Set Components

In [6]:
LLM.valid_models

{'cohere': ['command-r', 'command-r-plus'],
 'anthropic': ['claude-3-haiku-20240307',
  'claude-3-sonnet-2024022',
  'claude-3-opus-20240229'],
 'openai': ['gpt-4-turbo-preview',
  'gpt-4-0125-preview',
  'gpt-4-1106-preview',
  'gpt-4',
  'gpt-4-0613',
  'gpt-3.5-turbo',
  'gpt-3.5-turbo-1106',
  'gpt-3.5-turbo-0125']}

In [7]:
client = get_weaviate_client()
collection_name = 'Huberman_minilm_256'
reranker= ReRanker()
llm = load_azure_openai()
eval_gpt4 = 'gpt-4-0125-preview'

turbo_eval_model = CustomAzureOpenAI('gpt-35-turbo')
azure_eval_model = CustomAzureOpenAI('gpt-4')
anthro_eval_model = CustomAnthropic(model='claude-3-sonnet-20240229')

### Define TestCase acquisition functions

In [8]:
async def aget_actual_outputs(user_messages: list[str]):
    tasks = [llm.achat_completion(huberman_system_message, user_message, temperature=1.0) for user_message in user_messages]
    responses = await asyncio.gather(*tasks)
    return responses

In [9]:
async def acreate_test_cases( queries: list[str],
                              client: WeaviateWCS,
                              collection_name: str,
                              llm: LLM,
                              ) -> list[LLMTestCase]:
    '''
    Creates a list of LLM Test Cases based on query retrievals. 
    '''
    results = [client.hybrid_search(query, collection_name, limit=200) for query in tqdm(queries, 'QUERIES')]
    reranked = [reranker.rerank(result, queries[i], top_k=3) for i, result in enumerate(tqdm(results, 'RERANKING'))]
    user_messages = [generate_prompt_series(queries[i], rerank, 1) for i, rerank in enumerate(reranked)]
    actual_outputs = await aget_actual_outputs(user_messages)
    retrieval_contexts = [create_context_blocks(rerank) for rerank in reranked]
    test_cases = [LLMTestCase(input=input, actual_output=output, retrieval_context=context) \
                  for input, output, context in list(zip(queries, actual_outputs, retrieval_contexts))]
    return test_cases

### Get Test Cases

In [79]:
fake_queries = ['Who is the Grand High Poo-baa', 'Is Samuel L. Jackson a Christian, and if so which denomination', "What is Andy Huberman's middle name"]
fridman = ['Who is Lex Fridman']

In [13]:
fake_test_cases = await acreate_test_cases(fake_queries, client, collection_name, llm)

RERANKING: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.53it/s]


In [80]:
test_cases = await acreate_test_cases(fridman, client, collection_name, llm)

RERANKING: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.24it/s]


In [81]:
test_cases[0].actual_output

'Lex Fridman is an expert in electrical and computer engineering, artificial intelligence, and robotics. He hosts the Lex Fridman podcast, which covers a wide range of topics including technology, science, sports such as Brazilian Jiu-Jitsu, and mental health. He is known for engaging with various guests, spanning from scientists and engineers to comedians and friends, showcasing a diverse range of subjects and perspectives on his podcast. Additionally, Lex Fridman has been described as a thought leader who seeks to share the human experience and broaden knowledge for the benefit of all.'

In [12]:
#convert to list of dicts
test_case_list = [test_case.__dict__ for test_case in test_cases]

In [13]:
# #save results to disk
FileIO.save_as_json('../data/test_cases_50_256.json', test_case_list)
# test_cases = FileIO.load_json('../data/test_cases_256.json')
# test_cases = [LLMTestCase(input=tc['input'], actual_output=tc['actual_output'], retrieval_context=tc['retrieval_context'])
#               for tc in test_cases]

2024-05-01 00:28:06.071 | INFO     | src.preprocessor.preprocessing:save_as_json:111 - Data saved as json file here: ../data/test_cases_50_256.json


### Launch Testing

In [17]:
from src.evaluation.llm_evaluation import load_eval_response

In [55]:
async def asingle_eval_call(test_case: LLMTestCase,
                            model: str | DeepEvalBaseLLM,
                            metric: AnswerCorrectnessMetric,
                            return_context_data: bool=True
                            ) -> EvalResponse:
    if metric == AnswerCorrectnessMetric:
        metric = AnswerCorrectnessMetric(model)
    await metric.a_measure(test_case)
    response = load_eval_response(metric, test_case, return_context_data)
    return response

In [56]:
# gpt4_answers = await aiometer.run_on_each(asingle_eval_call, test_cases, max_per_second=1)

In [57]:
async def asystem_evaluation(test_cases: list[LLMTestCase],
                             model: DeepEvalBaseLLM,
                             metric: AnswerCorrectnessMetric,
                             return_context_data: bool=True
                             ):
    tasks = [asingle_eval_call(case, model, metric, return_context_data) for case in test_cases]
    responses = await asyncio.gather(*tasks)
    return responses

In [58]:
azure_evaluations_extra25 = await asystem_evaluation(test_cases, azure_eval_model, AnswerCorrectnessMetric)

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [23]:
%%time
azure_evaluations_first25 = evaluate(test_cases[:25], [acm], print_results=False)

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Output()

Output()

Output()

...

Output()

Output()

Output()

✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

CPU times: user 6.44 s, sys: 572 ms, total: 7.01 s
Wall time: 3min 7s


In [124]:
# cost = [eval.metrics[0].evaluation_cost for eval in azure_evaluations]
# sum(cost)

In [59]:
scores = [eval.score for eval in azure_evaluations_extra25]
sum(scores)/len(scores)

0.8360000000000001

In [34]:
goldens_first25 = [eval for eval in azure_evaluations_first25 if eval.metrics[0].score >= 0.9]

In [52]:
goldens_last25 = [eval for eval in azure_evaluations_last25 if eval.score >= 0.9]

In [62]:
goldens_extra25 = [eval for eval in azure_evaluations_extra25 if eval.score >= 0.9][:5]

In [63]:
len(goldens_first25), len(goldens_last25), len(goldens_extra25)

(7, 13, 5)

In [65]:
goldens_first25.extend(goldens_extra25)

In [70]:
golden_dicts = [eval.__dict__ for eval in goldens_first25]

In [72]:
golden_dicts = [{k:v for k,v in d.items() if k in ['input', 'actual_output', 'retrieval_context']} for d in golden_dicts]

In [77]:
import json
with open('../data/golden_datasets/llm_eval_testcases_initial.json', 'w') as f:
    json.dump(golden_dicts, f)

Bad pipe message: %s [b'\x01+W\x0bP\x8c\xfbjP\x1f0\xdeWG\x0b\x0c\x12\xd9 Bj\xfd\xbc\x88\xb3\x95\xda\xcdS_\x12IZ\xd2\xb3 8k/A\x7f\xeegc\xb2\xcf\xd5<A7\x15\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 @\x9fx\x7f\x08\x9c%\xba\xa8\xacH\x10K2t\xd8\xe5\xd1[\x0e']
Bad pipe message: %s [b'\xb9\x1c\xc0\xb7\xfa6{H\x00']
Bad pipe message: %s [b'"\xbf==*\xcf\xab\x9d\xb4)TN\xb3\x18\xd6\xd0\xe7\\\x00\x00>\xc0']
Bad pipe message: %s [b'\n\x009\x008\x007\x006\xc0\x0f\xc0\x05\x005\xc0\x13\xc0\t']
Bad pipe message: %s [b'6nL\x95fX\x83\xf0\xfc\xd2/\xf4\xb5\x9dN\x0c=\x8d\x00\x00\xa2\xc0\x14\xc0\n\x009\x008\x007\x006\x0

In [127]:
something = []
for eval in azure_evaluations:
    adict = eval.__dict__
    metrics = adict['metrics'][0].__dict__
    adict['metrics'] = metrics
    something.append(adict)

In [128]:
for d in something:
    del d['metrics']['evaluation_params']

In [129]:
FileIO.save_as_json('./azure_eval_results25.json', something,overwrite=True)

2024-04-30 04:20:03.722 | INFO     | src.preprocessor.preprocessing:save_as_json:111 - Data saved as json file here: ./azure_eval_results25.json


In [134]:
from rich import print

for res in azure_evaluations:
    if res.metrics['score'] < 0.9:
        print(res.metrics)
        print(res.input)
        print(res.actual_output)
        print(res.retrieval_context)
        print('\n')
        print('*'*100)
        print('\n')

{
    'name': 'AnswerCorrectness',
    'criteria': None,
    'using_native_model': False,
    'evaluation_model': 'gpt-4',
    'evaluation_steps': [
        'Compare the actual output with the retrieval context to verify factual accuracy.',
        'Assess if the actual output effectively addresses the specific information requirement stated in the 
input.',
        'Determine the comprehensiveness of the actual output in addressing all key aspects mentioned in the 
input.',
        'Score the actual output between 0 and 10, based on the accuracy and comprehensiveness of the information 
provided, with 10 being exactly correct and 0 being completely incorrect.',
        'If there is not enough information in the retrieval context to correctly answer the input, and the actual 
output indicates that the input cannot be answered with the provided context, then return a score of 10.'
    ],
    '_threshold': 0.5,
    'strict_mode': False,
    'async_mode': True,
    'evaluation_cost': None,
    'reason': "The actual output reflects relevant portions of the retrieval context, providing factual data 
regarding the prevalence of Adderall and Ritalin use among young people without an ADHD diagnosis. However, the 
output may slightly overstate the figures from the context by including the word 'upwards,' implying certainty of a
minimum, and integrating the 80% statistic without clear attribution to a reputable survey, which could affect the 
perceived accuracy of the information. This results in a score that reflects high accuracy and comprehensiveness, 
but not perfection due to potential overemphasis and lack of clear source attribution for the 80% figure.",
    'score': 0.8,
    'success': True
}

How prevalent is the use of Adderall and Ritalin without diagnosis of ADHD among young people?

The use of Adderall and Ritalin without a diagnosis of ADHD is prevalent among young people. According to the 
transcripts, it is mentioned that "in upwards of 25% of young people are taking things like Adderall despite not 
having a clinical diagnosis of ADHD." Additionally, it's stated that "as high as 80% of college age young adults 
have used one or several of prescription drugs such as Adderall, Ritalin, Vyvanse, or similar at some point and are
doing so without a prescription." This indicates a significant prevalence of off-prescription use of these 
medications among young individuals.

[
    '\nShow Summary: In episode 37 of the Huberman Lab podcast, Andrew Huberman discusses ADHD and methods for 
improving focus, both for those with ADHD and those without. He emphasizes that focus and the ability to ignore 
distractions are two distinct abilities, and both can be enhanced through various interventions. Huberman cautions 
against self-diagnosing ADHD, as many people may experience symptoms without meeting the full clinical criteria. He
explores the genetic components of ADHD, its independence from intelligence, and the tendency for people with ADHD 
to have hyperfocus on subjects of intense interest.\n\nHuberman delves into the role of dopamine in attention and 
motivation, its impact on specific brain circuits, and the "low dopamine hypothesis" of ADHD. He also discusses the
use of stimulant medications like Ritalin and Adderall, which are chemically similar to street drugs but can be 
beneficial for ADHD when used responsibly under medical supervision.\n\nDietary interventions, particularly the 
reduction of simple sugars and the implementation of an oligoantigenic\nShow Guest: Andrew Huberman\nTranscript: I 
just want to emphasize how these drugs work, some of the things that they do to enhance cognition and focus that 
actually serve the brain well in certain individuals and how they can be very detrimental in other individuals. I 
sort of blew right past it, but the fact that in upwards of 25% of young people are taking things like Adderall 
despite not having a clinical diagnosis of ADHD. Well, that\'s a ridiculously high number. A few years ago, it was 
estimated that Adderall use and Ritalin use without diagnosis of ADHD was second in incident only to cannabis. But 
actually now the consumption of Adderall without prescription is higher than the consumption of cannabis in that 
age group. So what that means is that there\'s a lot of stimulant use in that age group. And there are a lot of 
adults also using and abusing stimulants in order to gain focus. Now we could have a whole discussion about whether
or not life is becoming more demanding, whether or not the need for focus is excessive, and that\'s why people are 
doing that. Frankly, it\'s an interesting discussion, but it\'s not one that would deliver us to any 
answers.\n------------------------\n',
    '\nShow Summary: The Huberman Lab Podcast episode "Adderall, Stimulants & Modafinil for ADHD: Short- & 
Long-Term Effects" hosted by Andrew Huberman, a professor at Stanford School of Medicine, provides a comprehensive 
look at stimulants used to treat ADHD. The discussion covers the mechanisms of action of common ADHD medications 
like Adderall, Ritalin, Vyvanse, and non-stimulant options such as modafinil. Huberman addresses concerns about 
these drugs\' potential for addiction, psychotic symptoms, and long-term effects on brain function and physical 
health, including the impact on height and hormonal health. He also acknowledges the widespread off-prescription 
use of these medications among college students. The episode dives into the neurobiology of ADHD, explaining how 
these medications can paradoxically reduce hyperactivity by enhancing the prefrontal cortex\'s ability to regulate 
attention and suppress distractions. Huberman emphasizes the importance of proper diagnosis and treatment, 
including the potential benefits of medication-induced neuro\nShow Guest: Andrew Huberman\nTranscript: And of 
course, I will talk about the scientific literature surrounding the most frequently asked question about these 
compounds, which is what are the long-term consequences of taking any of them in childhood or in adulthood? Now, 
today\'s discussion centers around the use of these compounds both for childhood and for adult ADHD. But of course,
I\'d be remiss if I didn\'t acknowledge that there are a tremendous number of people that use these prescription 
drugs without a prescription in order to improve their ability to focus and indeed also use the

****************************************************************************************************

...

{
    'name': 'AnswerCorrectness',
    'criteria': None,
    'using_native_model': False,
    'evaluation_model': 'gpt-4',
    'evaluation_steps': [
        'Compare the actual output with the retrieval context to verify factual accuracy.',
        'Assess if the actual output effectively addresses the specific information requirement stated in the 
input.',
        'Determine the comprehensiveness of the actual output in addressing all key aspects mentioned in the 
input.',
        'Score the actual output between 0 and 10, based on the accuracy and comprehensiveness of the information 
provided, with 10 being exactly correct and 0 being completely incorrect.',
        'If there is not enough information in the retrieval context to correctly answer the input, and the actual 
output indicates that the input cannot be answered with the provided context, then return a score of 10.'
    ],
    '_threshold': 0.5,
    'strict_mode': False,
    'async_mode': True,
    'evaluation_cost': None,
    'reason': "The output is factually accurate by stating that there are no current FDA-approved medications for 
the core features of autism, aligning well with the retrieval context which echoes this sentiment. It also 
addresses the initial question's concern about the importance of having such medications. However, the output could
be slightly more comprehensive by mentioning ongoing research or indicating potential developments, which can be 
inferred from the retrieval context discussing ongoing research into novel treatments. This would offer a more 
complete picture in response to the question about the importance of approved medications.",
    'score': 0.8,
    'success': True
}

Why is it important to have medications approved to treat the core features of autism?

It is important to have medications approved to treat the core features of autism because currently, there are only
two drugs approved by the FDA to treat autism, and they are both antipsychotics. These drugs treat associated 
features like irritability, but they have off-target effects such as weight gain. There are no medications 
currently approved in the U.S. or anywhere else for that matter to treat the core features of autism. As a result, 
there is a significant need for medications specifically approved to target the core symptoms of autism, such as 
social interaction deficits, in order to provide more effective and targeted treatments for individuals with autism
spectrum disorder.

[
    "\nShow Summary: In the Huberman Lab podcast episode with Dr. Karen Parker, they discuss the science of autism,
exploring the biological basis of social functioning and its perturbations in autism spectrum disorders (ASD). Dr. 
Parker's research focuses on understanding autism's causes and developing new treatments. They address the 
increased incidence of autism diagnoses, which is attributed to better detection and genuine prevalence growth. Dr.
Parker presents new findings suggesting novel causes and treatments for autism.\n\nThe conversation delves into the
challenges of diagnosing autism at early stages, given its reliance on behavioral assessment and the heterogeneity 
of the disorder. They discuss the genetic and environmental factors contributing to autism, highlighting the need 
for personalized approaches to treatment due to the condition's complexity. The podcast also touches upon the 
potential role of oxytocin and vasopressin, neuropeptides involved in social behaviors, in the context of 
autism.\n\nDr. Parker emphasizes the importance of sophisticated models for studying autism and the limitations of 
using traditional animal models like mice.\nShow Guest: Dr. Karen Parker\nTranscript: Are parents of autistic kids 
able to like buy oxytocin nasal spray? No. So it would need to be written, like the prescription would need to be 
written by a physician. And it's not on the market, right? So there's one thing we should say is there's only two 
drugs that are approved by the FDA to treat autism, and they're both antipsychotics, which they treat associated 
features like irritability, and they have off-target effects like weight gain. So we don't have any medications 
that are currently approved in the U.S. or anywhere else for that matter to treat the core features of autism. 
Interesting and unfortunate. And hopefully that will change in the not-too-distant future. Do we know that children
with autism, people with autism, because I'm going to just sort of assume that autism is stable over the lifespan. 
Like if a child is diagnosed with autism, are they going to be an adolescent and adult with autism? So I would say 
that in a lot of cases, autism has lifelong impact. But there are people who outgrow their diagnosis. You know, 
there are people who respond well to behavioral therapy.\n------------------------\n",
    '\nShow Summary: In the nucleus accumbens, a key brain region for reward processing, Dr. Robert Malenka\'s 
research found that oxytocin enhances the release of serotonin, which is another neuromodulator involved in mood 
and social behavior. This finding suggests that oxytocin may play a role in promoting sociability by influencing 
serotonin levels within the reward circuitry. Oxytocin, often referred to as the "love hormone," has been 
previously linked to pair bonding and social behaviors in various species, like the prairie vole. Malenka\'s work 
explores how positive, pro-social, non-aggressive interactions are highly reinforcing and likely involve the reward
circuitry, potentially providing evolutionary advantages in survival, reproduction, and protection from predators. 
While oxytocin\'s role in pair bonding has been debated, Malenka\'s research contributes to the understanding of 
its function in sociability and its interaction with other neuromodulators like serotonin in the context of the 
reward system.\nShow Guest: Dr. Robert Malenka\nTranscript: I mean, I think that returning to the issue of autism 
and the role of serotonin, so in autism, there seems to be less of a reinforcement pathway for certain kinds of 
social interactions in some individuals with autism. And I\'m aware that there are some prescription treatments for
autism that capitalize on the serotonergic system and dopamine system. So is it fentamine? To my knowledge, the 
only FDA-approved pharmacologic therapeutic for individuals with autism spectrum disorder is actually, oh God, I\'m
just blanking. It\'s not a seroton

****************************************************************************************************

In [9]:
azure_evaluations_first25

NameError: name 'azure_evaluations_first25' is not defined